# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Мартиросян Завен Карапетович`

Группа: `DS - 11
`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [3]:
pip install OSMPythonTools

  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.2.9-py3-none-any.whl size=12773 sha256=782259e7a6f947452ace2840e5994a831aac522833ba950ef37bcc3c52c39e8e
  Stored in directory: /root/.cache/pip/wheels/fe/26/85/92c2a154eeda3c9ebf8ffc53b685a697bd9c1a182260c9a030
  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8969 sha256=fc73ec681a9cfb5aaf29430f41548fdf770717fd38d6d444528129b99e88be08
  Stored in directory: /root/.cache/pip/wheels/9c/e8/b6/5009ec756a2f40eed690a8b0a95549cd788c1ec968832876df
Successfully built OSMPythonTools pytest-sugar
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import folium
import json 
import geopandas as gpd

In [4]:
import shapely
from shapely.geometry import Point
from shapely.geometry import Polygon

from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim

In [5]:
!wget https://www.dropbox.com/s/w0t3944ljho8j1y/poly_afg.txt

--2020-11-04 19:37:05--  https://www.dropbox.com/s/w0t3944ljho8j1y/poly_afg.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/w0t3944ljho8j1y/poly_afg.txt [following]
--2020-11-04 19:37:06--  https://www.dropbox.com/s/raw/w0t3944ljho8j1y/poly_afg.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7270cbbacc436fbac3094583ae.dl.dropboxusercontent.com/cd/0/inline/BCkPPUq4_f1niNSCrRu7tFpTmF4BNrobVbEfFbn8sMSFnUB5qzowtQ5unZJYSjC7TOO1ldi7kbfeavDwQZHXsGgQoMuuUox8OfSUgzJrriGcDNglH6SGF-6iVJn7z8BlukI/file# [following]
--2020-11-04 19:37:06--  https://uc7270cbbacc436fbac3094583ae.dl.dropboxusercontent.com/cd/0/inline/BCkPPUq4_f1niNSCrRu7tFpTmF4BNrobVbEfFbn8sMSFnUB5qzowtQ5unZJYSjC7TOO1ldi7kbfeavDwQZHXsGgQoMuuUox8OfSUgzJrriGcDNglH6

In [7]:
f = open("poly_afg.txt")

afg_pol = []
while f.readline():
  coords = f.readline().split()
  afg_pol.append(Point(float(coords[0]), float(coords[1])))
afg_pol = Polygon(afg_pol)

gdf = gpd.GeoSeries(afg_pol, crs = "EPSG:4326")

In [8]:
m = folium.Map(
    location = afg_pol.centroid.coords[:][0][::-1],
    zoom_start = 2
)
folium.GeoJson(gdf.geometry).add_to(m)
folium.LayerControl().add_to(m)
m

In [9]:
import random
random.seed(21)           # воспроизводимость

x_min, y_min = -180, -180 # изначальная граница - весь мир
x_max, y_max = 180, 180   # изначальная граница - весь мир

new_x = random.uniform(x_min, x_max)
new_y = random.uniform(y_min, y_max)

new_point = Point(new_x, new_y)
folium.Marker(location=[new_y, new_x]).add_to(m)

while afg_pol.contains(new_point) == False:

  if x_min / 2  < afg_pol.centroid.coords[0][0]:
    x_min = x_min / 2

  if x_max / 2  > afg_pol.centroid.coords[0][0]:
    x_max = x_max / 2

  if y_min / 2  < afg_pol.centroid.coords[0][1]:
    y_min = y_min / 2
  if y_max / 2  > afg_pol.centroid.coords[0][1]:
    y_max = y_max / 2
  
  new_x = random.uniform(x_min, x_max)
  new_y = random.uniform(y_min, y_max)
  new_point = Point(new_x, new_y)

  folium.Marker(location=[new_y, new_x]).add_to(m)

In [10]:
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [12]:
pip install openrouteservice

Note: you may need to restart the kernel to use updated packages.


In [13]:
import json
import requests

import folium
import pyproj
from pyproj import Transformer
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

from openrouteservice import client

In [14]:
url = 'https://data.cityofnewyork.us/api/views/a3nt-yts4/rows.json'

def CreateBufferPolygon(point_in, resolution=10, radius=10):

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632') # WGS84 UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 10 m buffer

    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs

In [17]:
from functools import partial
from pyproj import Proj, transform
from pyproj import CRS

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
# Set up the fundamentals
api_key = '5b3ce3597851110001cf6248310884f9dd72498d96b00421fe1d2e0b' # Individual api key
clnt = client.Client(key = api_key) # Create client with api key
ny_school_json = requests.get(url).json() # Get data as JSON

# print(ny_school_json)

map_params = {'tiles': 'Stamen Toner',
              'location': ([40.7, -73.99]),
              'zoom_start': 12}
map1 = folium.Map(**map_params)
site_coords = []
sites_poly = []
sites_poly_coords = []

for school in ny_school_json["data"]:

    yx = school[8].split()
    x = float(yx[2].strip("()"))
    y = float(yx[1].strip("()"))
    site_coords.append([y, x])

for coords in site_coords:
    folium.features.Marker(list(reversed(coords)),
                           popup = 'School<br>{0}'.format(coords)).add_to(map1)#.format(coords)


    # Create buffer polygons around schools with 140 m radius and low resolution
    site_poly_coords = CreateBufferPolygon(coords,
                                           resolution = 4, # low resolution to keep polygons lean
                                           radius = 100)
    folium.vector_layers.Polygon(site_poly_coords).add_to(map1)
    sites_poly.append(site_poly_coords)

    site_poly_coords = [(y, x) for x, y in site_poly_coords] # Reverse coords for folium/Leaflet
    folium.vector_layers.Polygon(locations = site_poly_coords,
                                 color = '#ffd699',
                                 fill_color = '#ffd699',
                                 fill_opacity = 0.2,
                                 weight = 3).add_to(map1)

In [20]:
def style_function(color):
    return lambda feature: dict(color = color,
                              weight = 3,
                              opacity = 0.5)

In [24]:
# Create new map to start from scratch
map_params.update({'location': ([40.75, -73.98]),
                   'zoom_start': 13})
#map1 = folium.Map(map_params)

# Request normal route between appropriate locations without construction sites
request_params = {'coordinates': [[-73.97228, 40.76561],
                                 [-73.98959, 40.70325]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}
route_normal = clnt.directions(**request_params)
folium.features.GeoJson(data = route_normal,
                        name = 'Route without schools',
                        style_function = style_function('#FF0000'),
                        overlay = True).add_to(map1)

In [25]:
# Buffer route with 0.01 degrees
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.0013)
#folium.features.GeoJson(data = geometry.mapping(route_buffer),
 #                       name = 'Route Buffer',
  #                      style_function = style_function('#FFFFFF'),
   #                     overlay = True).add_to(map1)

# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map1)

        folium.vector_layers.PolyLine(site_poly).add_to(map1)
        folium.vector_layers.Polygon(site_poly,
                                 color = '#dddd00',
                                 fill_color = '#dddd00',
                                 fill_opacity = 0.8,
                                 weight = 5).add_to(map1)
        sites_buffer_poly.append(poly)

In [ ]:
# Buffer route with 0.01 degrees
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.0013)
#folium.features.GeoJson(data = geometry.mapping(route_buffer),
 #                       name = 'Route Buffer',
  #                      style_function = style_function('#FFFFFF'),
   #                     overlay = True).add_to(map1)

# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map1)

        folium.vector_layers.PolyLine(site_poly).add_to(map1)
        folium.vector_layers.Polygon(site_poly,
                                 color = '#dddd00',
                                 fill_color = '#dddd00',
                                 fill_opacity = 0.8,
                                 weight = 5).add_to(map1)
        sites_buffer_poly.append(poly)
# Add the site polygons to the request parameters
request_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))}
route_detour = clnt.directions(**request_params)

folium.features.GeoJson(data=route_detour,
                        name='Route without schools',
                        style_function=style_function('#00FF00'),
                        overlay=True).add_to(map1)

detour_buffer = LineString(route_detour['features'][0]['geometry']['coordinates']).buffer(0.0013)
folium.features.GeoJson(data = geometry.mapping(detour_buffer),
                        name = 'Route Buffer',
                        style_function = style_function('#FFFF00'),
                        overlay = True).add_to(map1)

# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if detour_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map1)
        sites_buffer_poly.append(poly)

map1.add_child(folium.map.LayerControl())


map1